In [ ]:
using CSV
using DataFrames
using Dates
using Statistics
using LinearAlgebra
using JuMP
using Gurobi

data_dir = "./data"

# ─── HELPERS ───────────────────────────────────────────────────────────────────

"""
    compute_returns(prices::Vector{Float64})

From a sorted price series returns the vector of simple returns r_t = (P_t/P_{t-1}) - 1
"""
function compute_returns(prices::Vector{Float64})
    return diff(prices) ./ prices[1:end-1]
end

# ─── LOAD PRICES & BUILD RETURNS DATAFRAME ────────────────────────────────────

# 1. Read each CSV into a (Date, Return) DataFrame
# 2. Left-join them all on Date so we have one DataFrame with columns: Date, r1, r2, r3, …

dfs = DataFrame[]
for file in filter(f->endswith(lowercase(f), ".csv"), readdir(data_dir; join=true))
    df = CSV.read(file, DataFrame)
    df.Date = Date.(df.Date, dateformat"u d, Y")
    sort!(df, :Date)
    rs = compute_returns(df.Close)
    # drop the first date (no return for it)
    df = df[2:end, [:Date]]
    df.:return = rs
    push!(dfs, df)
end

# give each return‐column a unique name, e.g. :r1, :r2, …
for (i, df) in enumerate(dfs)
    rename!(df, :return => Symbol("r$i"))
end

# join all on Date
rets = reduce((a,b)->innerjoin(a, b, on=:Date), dfs)

# ─── COMPUTE μ AND Σ ──────────────────────────────────────────────────────────

# expected return vector μ
μ = [mean(rets[!, Symbol("r$i")]) for i in 1:length(dfs)]

# covariance matrix Σ
# make a matrix of returns: each column is one asset
R = Matrix(rets[:, Not(:Date)])
Σ = Symmetric(cov(R, dims=1))  # cov along rows, returns a pxp matrix

println("μ = ", μ)
println("Σ = ")
display(Σ)



μ = [0.002592239832991949, 0.004611705992668354, 0.0014443258538598808]
Σ = 


3×3 Symmetric{Float64, Matrix{Float64}}:
 0.0035613    0.00139997   0.000828436
 0.00139997   0.00197079   0.000601053
 0.000828436  0.000601053  0.00124855

In [ ]:
using JuMP, Gurobi
using LinearAlgebra

"""
    objective(w, w_prev, Σ, μ, γ, L, λ)

Compute

    lamda wᵀ sigma w  -  (1-lamda) wᵀ mu  +  ∑ᵢ gamma[i] * |w[i] - w_prev[i]| / L[i]

Inputs
- `w`       : current weight vector, size n
- `w_prev`  : previous weight vector, size n
- `sigma`       : n×n covariance matrix
- `mu`       : n-vector of expected returns
- `gamma`       : n-vector of sparsity penalties
- `L`       : n-vector of scaling factors
- `lamda`       : scalar trade‐off parameter in [0,1]
"""

# — Example usage of objective() directly —
n = 3
sigma      = Symmetric(randn(n,n))    # example covariance
gamma      = abs.(randn(n))
L      = rand(n) .+ 0.1          # ensure nonzero
w_prev = randn(n)
w      = randn(n)
lamda      = 0.5

model = Model(Gurobi.Optimizer)
@variable(model, w[1:n])
# (you can add constraints here, e.g. sum(w) == 1, w .>= 0, etc.)
@objective(model, Min,
    lamda * w' * sigma * w
  - (1 - lamda) * dot(w, μ)
  + sum( gamma[i] * abs(w[i] - w_prev[i]) / L[i] for i in 1:n )
)
optimize!(model)